We now demonstrate defining our own `Embedding` module, and using it in place of the predefined `Embedding` layer.

In [1]:
from fastai.tabular.all import *

class MyEmbedding(Module):
    def __init__(self, ni, nf):
        self.weight = nn.Parameter(torch.randn(ni, nf) * 0.1)

    def __call__(self, x):
        return self.weight[x]

With this defined, we now redo the work done in previous notebooks up to model definition.

In [2]:
from fastai.collab import *

path = untar_data(URLs.ML_100k)

ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0, 1), names=('movie_id', 'title'), header=None)
ratings = ratings.merge(movies)
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)

n_users  = len(dls.classes['user_id'])
n_movies = len(dls.classes['title'])
n_factors = 5

The model definition is as follows.

In [3]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = MyEmbedding(n_users, n_factors)
        self.user_bias = MyEmbedding(n_users, 1)
        self.movie_factors = MyEmbedding(n_movies, n_factors)
        self.movie_bias = MyEmbedding(n_movies, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)


And we now give it a spin.

In [4]:

model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.887599,0.995600,00:07
1,0.596472,0.912096,00:06
2,0.476899,0.897185,00:06
3,0.401480,0.880874,00:06
4,0.384160,0.876893,00:06
